
<center><h1>The Annotated Transformer</h1> </center>


<center>
<p><a href="https://arxiv.org/abs/1706.03762">Attention is All You Need
</a></p>
</center>

<img src="images/aiayn.png" width="70%"/>

* *v2022: Austin Huang, Suraj Subramanian, Jonathan Sum, Khalid Almubarak,
   and Stella Biderman.*
* *[Original](https://nlp.seas.harvard.edu/2018/04/03/attention.html):
   [Sasha Rush](http://rush-nlp.com/).*


The Transformer has been on a lot of
people's minds over the last <s>year</s> five years.
This post presents an annotated version of the paper in the
form of a line-by-line implementation. It reorders and deletes
some sections from the original paper and adds comments
throughout. This document itself is a working notebook, and should
be a completely usable implementation.
Code is available
[here](https://github.com/harvardnlp/annotated-transformer/).


# Prelims

<a href="#background">Skip</a>

In [1]:
# ! pip install GPUtil

In [2]:
# !pip install -r requirements.txt

In [3]:
# # Uncomment for colab
# #
# !pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
# !python3 -m spacy download de_core_news_sm
# !python3 -m spacy download en_core_web_sm

In [4]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import gc

# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [5]:
torch.cuda.empty_cache()

In [6]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

> My comments are blockquoted. The main text is all from the paper itself.

# Part 1: Model Architecture

# Model Architecture


Most competitive neural sequence transduction models have an
encoder-decoder structure
[(cite)](https://arxiv.org/abs/1409.0473). Here, the encoder maps an
input sequence of symbol representations $(x_1, ..., x_n)$ to a
sequence of continuous representations $\mathbf{z} = (z_1, ...,
z_n)$. Given $\mathbf{z}$, the decoder then generates an output
sequence $(y_1,...,y_m)$ of symbols one element at a time. At each
step the model is auto-regressive
[(cite)](https://arxiv.org/abs/1308.0850), consuming the previously
generated symbols as additional input when generating the next.

In [7]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [8]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)


The Transformer follows this overall architecture using stacked
self-attention and point-wise, fully connected layers for both the
encoder and decoder, shown in the left and right halves of Figure 1,
respectively.

![](images/ModalNet-21.png)

## MNet

In [9]:
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim

import time
import math
m_shape = (512, 512) # EMB, EMB

class GradSaver_Compute_M(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, sequentialOutput, saver):
        # print('in forward in grad saver')
        ctx.save_for_backward(x, saver, sequentialOutput)
        return sequentialOutput.clone().detach()
    
    

    @staticmethod
    def backward(ctx, gradients):
        global lst_sqs_sum
        
        x, saver, s = ctx.saved_tensors
        g = gradients
        saver.g = gradients.clone()

        l,b,d = x.shape

        # reshape
        x = x.reshape(l*b, d)
        s = s.reshape(l*b, d)
        g = g.reshape(l*b, d)

        K = math.ceil(l*b / d)
        # pad_size = ( (K*d*d) - (b*l*d) ) // d # should be int
        pad_size = ( (K*d) - (b*l) ) # should be int
        pad_value = 0

        # now apply padding to x,s,g and reshape
        x_pad = torch.nn.functional.pad(x, (0,0,0,pad_size), 'constant', pad_value).reshape(K,d,d)
        s_pad = torch.nn.functional.pad(s, (0,0,0,pad_size), 'constant', pad_value).reshape(K,d,d)
        g_pad = torch.nn.functional.pad(g, (0,0,0,pad_size), 'constant', pad_value).reshape(K,d,d)

        delta = 1e-9
        perturbation = delta + torch.eye(d).to('cuda:1')
        saver.g = gradients.clone()
        # print('saver.g.dtype', saver.g.dtype)

        gbar = []

        # gbar = [torch.mm(g_pad[k,:,:], layerwisenorm_M(softmax_func_M(torch.linalg.lstsq(x_pad[k,:,:], s_pad[k,:,:]).solution))) for k in range(K)]
        gbar = [torch.mm(g_pad[k,:,:], layerwisenorm_M(softmax_func_M(torch.linalg.solve(x_pad[k,:,:] + perturbation, s_pad[k,:,:])))) for k in range(K)]

        g_out = torch.cat(gbar)
        # print(g_out.shape)

        # unpad and reshape
        g_out = g_out[:g_out.shape[0]-pad_size,:].reshape(l,b,d)
        # print('g_out.dtype', g_out.dtype)
        
        return g_out, None, None

class GradSaver_As_Is(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, sequentialOutput, saver):
        # print('in forward in grad saver')
        ctx.save_for_backward(x, saver, sequentialOutput)
        return sequentialOutput.clone().detach()

    @staticmethod
    def backward(ctx, gradients):
        global lst_sqs_sum
        
        x, saver, s = ctx.saved_tensors
        saver.g = gradients.clone()
        # print('in gradsaver backward')
        return gradients, None, None
    

class MNet_Transformer_As_Is(torch.nn.Module):
    def __init__(self, sequentialLayers, output_size):
        print('Creating MNet_Transformer')
        super(MNet_Transformer_As_Is, self).__init__()
        self.layers = []
        for l in sequentialLayers:
            # if not isinstance(l, nn.ReLU):
            if True: # insert all layers (attn, linear, dropout, norm)
                # l.requires_grad = True
                self.layers.append(l)
        # self.layers.requires_grad = False
        self.input_x = None
        self.layersOutput = []
        self.saver = torch.ones(output_size, dtype = self.layers[0].norm.a_2.data.dtype, requires_grad=True).to('cuda:1') # check dims of this (batch, output of M) 
        self.gradDiverge = GradSaver_As_Is.apply
        self.sequentialOutput = None


        # vars for custom_attn
        self.h = H
        self.d_k = MODEL_DIM // self.h
        self.linears = clones(nn.Linear(MODEL_DIM, MODEL_DIM), 4)
        self.dropout = nn.Dropout(p=0.1)
        self.self_attn_func = MultiHeadedAttention(H, MODEL_DIM)
        # self.custom_multihead_attn = self.CustomMultiHeadedAttention(H, MODEL_DIM)

        # Switch to enable M ---------- NOTE for hybrid method
        self.use_M = True

    def getLayersOutput(self, x, mask,):
        self.layersOutput = []

        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.input_x = x_clone
        else:
            self.input_x = x

        sublayer_ct = 0
        lo = self.input_x
        for l in self.layers:
            # print(l)
            # print(type(l))

            if 'sublayer' in str(type(l)).lower() and sublayer_ct == 0:
                # lo = l(lo, self.custom_multihead_attn(lo,lo,lo, mask))
                lo = l(lo, lambda x: self.self_attn_func(x, x, x, mask))
                self.attn = l.attn_mat
                sublayer_ct += 1
            elif 'sublayer' in str(type(l)).lower() and sublayer_ct > 0:
                lo = l(lo, feed_forward_layer)
                sublayer_ct = 0
            elif 'feed' in str(type(l)).lower():
                feed_forward_layer = l # save for second sublayer
            else:
                print('ERROR: unexpected layer: ', l)

        return lo

    def forward(self, x, mask,):
        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.sequentialOutput = self.getLayersOutput(x_clone, mask,)
            # self.saver = sequentialOutput.clone().detach()
            if self.saver.shape != self.sequentialOutput.shape:
                self.saver = torch.ones(self.sequentialOutput.shape).to('cuda:1')
            # print('self.sequentialOutput.requires_grad ', self.sequentialOutput.requires_grad)

            t = self.gradDiverge(x, self.sequentialOutput.clone().detach(), self.saver)
        else:
            self.sequentialOutput = self.getLayersOutput(x, mask,)
            t = self.sequentialOutput
        # print('t, ', t.grad)
        return t

    def backwardHidden(self):
        # print('in MNet, ', self.saver.grad)
        # print('seqoutput grad', self.sequentialOutput.grad)
        self.sequentialOutput.backward(gradient = self.saver.g.clone().detach())
        
    def get_parameters(self):
        ps = []
        ps.append(self.layers[0].out_proj.weight) # self attn
        ps.append(self.layers[0].out_proj.bias)

        for l in self.layers:
            if hasattr(l, 'weight') and hasattr(l, 'bias'):
                ps.append(l.weight)
                ps.append(l.bias)
        return ps
    
class MNet_Transformer_Compute_M(torch.nn.Module):
    def __init__(self, sequentialLayers, output_size):
        print('Creating MNet_Transformer')
        super(MNet_Transformer_Compute_M, self).__init__()
        self.layers = []
        for l in sequentialLayers:
            # if not isinstance(l, nn.ReLU):
            if True: # insert all layers (attn, linear, dropout, norm)
                # l.requires_grad = True
                self.layers.append(l)
        # self.layers.requires_grad = False
        self.input_x = None
        self.layersOutput = []
        self.saver = torch.ones(output_size, dtype = self.layers[0].norm.a_2.data.dtype, requires_grad=True).to('cuda:1') # check dims of this (batch, output of M) 
        self.gradDiverge = GradSaver_Compute_M.apply
        self.sequentialOutput = None


        # vars for custom_attn
        self.h = H
        self.d_k = MODEL_DIM // self.h
        self.linears = clones(nn.Linear(MODEL_DIM, MODEL_DIM), 4)
        self.dropout = nn.Dropout(p=0.1)
        self.self_attn_func = MultiHeadedAttention(H, MODEL_DIM)
        # self.custom_multihead_attn = self.CustomMultiHeadedAttention(H, MODEL_DIM)

        # Switch to enable M ---------- NOTE for hybrid method
        self.use_M = True

    def getLayersOutput(self, x, mask,):
        self.layersOutput = []

        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.input_x = x_clone
        else:
            self.input_x = x

        sublayer_ct = 0
        lo = self.input_x
        for l in self.layers:
            # print(l)
            # print(type(l))

            if 'sublayer' in str(type(l)).lower() and sublayer_ct == 0:
                # lo = l(lo, self.custom_multihead_attn(lo,lo,lo, mask))
                lo = l(lo, lambda x: self.self_attn_func(x, x, x, mask))
                self.attn = l.attn_mat
                sublayer_ct += 1
            elif 'sublayer' in str(type(l)).lower() and sublayer_ct > 0:
                lo = l(lo, feed_forward_layer)
                sublayer_ct = 0
            elif 'feed' in str(type(l)).lower():
                feed_forward_layer = l # save for second sublayer
            else:
                print('ERROR: unexpected layer: ', l)

        return lo

    def forward(self, x, mask,):
        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.sequentialOutput = self.getLayersOutput(x_clone, mask,)
            # self.saver = sequentialOutput.clone().detach()
            if self.saver.shape != self.sequentialOutput.shape:
                self.saver = torch.ones(self.sequentialOutput.shape).to('cuda:1')
            # print('self.sequentialOutput.requires_grad ', self.sequentialOutput.requires_grad)

            t = self.gradDiverge(x, self.sequentialOutput.clone().detach(), self.saver)
        else:
            self.sequentialOutput = self.getLayersOutput(x, mask,)
            t = self.sequentialOutput
        # print('t, ', t.grad)
        return t

    def backwardHidden(self):
        # print('in MNet, ', self.saver.grad)
        # print('seqoutput grad', self.sequentialOutput.grad)
        self.sequentialOutput.backward(gradient = self.saver.g.clone().detach())
        
    def get_parameters(self):
        ps = []
        ps.append(self.layers[0].out_proj.weight) # self attn
        ps.append(self.layers[0].out_proj.bias)

        for l in self.layers:
            if hasattr(l, 'weight') and hasattr(l, 'bias'):
                ps.append(l.weight)
                ps.append(l.bias)
        return ps

## Encoder and Decoder Stacks

### Encoder

The encoder is composed of a stack of $N=6$ identical layers.

In [10]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [11]:
class CustomSequential(nn.Module):
    def __init__(self, encoder_layers) -> None:
        super(CustomSequential, self).__init__()
        self.encoder_layers = encoder_layers
        modules = []
        for i in range(len(encoder_layers)):
            # modules.append(encoder_layers[i].self_attn)
            modules.append(encoder_layers[i].sublayer[0])
            modules.append(encoder_layers[i].feed_forward)
            modules.append(encoder_layers[i].sublayer[1])
            # try to add the layernorms too
        # print(modules)
        # print(len(modules))
        self.custom_sequential = nn.Sequential(*modules)

class Encoder_MNet_As_Is(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N, skipping_layer_idx, use_layers_M_inference=True):
        super(Encoder_MNet_As_Is, self).__init__()
        self.use_layers_M_inference = use_layers_M_inference
        self.layers = clones(layer, N)
        # self.layers = [layer for _ in range(N)]
        self.norm = LayerNorm(layer.size)
        
        # self.layer_idx_M = [3] # just choose one encoder layer to abstract
        # self.mnet = MNet_Transformer(CustomSequential([layer]).custom_sequential, (BATCH_SIZE, MODEL_DIM))
        
        
        # self.layer_idx_M = [2,3,4] # exclude input and output layers
        # self.mnet = MNet_Transformer(CustomSequential(self.layers[2:5]).custom_sequential, (BATCH_SIZE, MODEL_DIM))

        self.layer_idx_M1 = skipping_layer_idx
        # self.layer_idx_M2 = [3,4,5]
        self.mnet1 = MNet_Transformer_As_Is(CustomSequential(self.layers[skipping_layer_idx[0]:]).custom_sequential, (BATCH_SIZE, MODEL_DIM))
        # self.mnet2 = MNet_Transformer(CustomSequential(self.layers[4:]).custom_sequential, (BATCH_SIZE, MODEL_DIM))

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for idx, layer in enumerate(self.layers):
            if idx == self.layer_idx_M1[0]:
                if self.use_layers_M_inference:
                    x = self.mnet1(x, mask)
                # else continue to next layer
            elif idx in self.layer_idx_M1:

                continue # already covered by the MNet
            else:
                x = layer(x, mask)
        return self.norm(x)
    

class Encoder_MNet_Compute_M(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N, skipping_layer_idx, use_layers_M_inference=True):
        super(Encoder_MNet_Compute_M, self).__init__()
        self.use_layers_M_inference = use_layers_M_inference
        self.layers = clones(layer, N)
        # self.layers = [layer for _ in range(N)]
        self.norm = LayerNorm(layer.size)
        
        # self.layer_idx_M = [3] # just choose one encoder layer to abstract
        # self.mnet = MNet_Transformer(CustomSequential([layer]).custom_sequential, (BATCH_SIZE, MODEL_DIM))
        
        
        # self.layer_idx_M = [2,3,4] # exclude input and output layers
        # self.mnet = MNet_Transformer(CustomSequential(self.layers[2:5]).custom_sequential, (BATCH_SIZE, MODEL_DIM))

        self.layer_idx_M1 = skipping_layer_idx
        # self.layer_idx_M2 = [3,4,5]
        self.mnet1 = MNet_Transformer_Compute_M(CustomSequential(self.layers[skipping_layer_idx[0]:]).custom_sequential, (BATCH_SIZE, MODEL_DIM))
        # self.mnet2 = MNet_Transformer(CustomSequential(self.layers[4:]).custom_sequential, (BATCH_SIZE, MODEL_DIM))

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for idx, layer in enumerate(self.layers):
            if idx == self.layer_idx_M1[0]:
                if self.use_layers_M_inference:
                    x = self.mnet1(x, mask)
                # else continue to next layer
            elif idx in self.layer_idx_M1:

                continue # already covered by the MNet
            else:
                x = layer(x, mask)
        return self.norm(x)
    
class Encoder_Base(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N, use_layers_M_inference=True):
        super(Encoder_Base, self).__init__()
        self.use_layers_M_inference = use_layers_M_inference
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

        self.layer_idx_M1 = [3,4,5]

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for idx, layer in enumerate(self.layers):
            if idx in self.layer_idx_M1 and not self.use_layers_M_inference:
                # print('skipping layer idx')
                continue # skip these layers for logic segmentation experiment
            else:
                # print(layer)
                x = layer(x, mask)
        # print('DONE')
        return self.norm(x)


We employ a residual connection
[(cite)](https://arxiv.org/abs/1512.03385) around each of the two
sub-layers, followed by layer normalization
[(cite)](https://arxiv.org/abs/1607.06450).

In [12]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()

        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        try:
            return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
        except RuntimeError:
            return self.a_2.cuda('cuda:1') * (x - mean) / (std + self.eps) + self.b_2.cuda('cuda:1')
        
class LayerNorm_CUDA(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm_CUDA, self).__init__()

        self.a_2 = nn.Parameter(torch.ones(features)).to('cuda:1')
        self.b_2 = nn.Parameter(torch.zeros(features)).to('cuda:1')
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)

        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


In [13]:
softmax_func_M = nn.Softmax(dim=-1)
layerwisenorm_M = LayerNorm_CUDA(m_shape)


That is, the output of each sub-layer is $\mathrm{LayerNorm}(x +
\mathrm{Sublayer}(x))$, where $\mathrm{Sublayer}(x)$ is the function
implemented by the sub-layer itself.  We apply dropout
[(cite)](http://jmlr.org/papers/v15/srivastava14a.html) to the
output of each sub-layer, before it is added to the sub-layer input
and normalized.

To facilitate these residual connections, all sub-layers in the
model, as well as the embedding layers, produce outputs of dimension
$d_{\text{model}}=512$.

In [14]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        self.attn_mat = None

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."

        imd = sublayer(self.norm(x))
        if type(imd) is tuple:
            self.attn_mat = imd[-1] # grab second output from MHAttn
        else:
            imd = [imd, -1]
        return x + self.dropout(imd[0]) # regular opeartion


Each layer has two sub-layers. The first is a multi-head
self-attention mechanism, and the second is a simple, position-wise
fully connected feed-forward network.

In [15]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder

The decoder is also composed of a stack of $N=6$ identical layers.


In [16]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)


In addition to the two sub-layers in each encoder layer, the decoder
inserts a third sub-layer, which performs multi-head attention over
the output of the encoder stack.  Similar to the encoder, we employ
residual connections around each of the sub-layers, followed by
layer normalization.

In [17]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)


We also modify the self-attention sub-layer in the decoder stack to
prevent positions from attending to subsequent positions.  This
masking, combined with fact that the output embeddings are offset by
one position, ensures that the predictions for position $i$ can
depend only on the known outputs at positions less than $i$.

In [18]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

In [19]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

![](images/ModalNet-20.png)

In [20]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None # this is the output of softmax (before multiplied by V) ------------------ NOTE
        # self.attn
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x), self.attn

## Position-wise Feed-Forward Networks

In addition to attention sub-layers, each of the layers in our
encoder and decoder contains a fully connected feed-forward network,
which is applied to each position separately and identically.  This
consists of two linear transformations with a ReLU activation in
between.

$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$

While the linear transformations are the same across different
positions, they use different parameters from layer to
layer. Another way of describing this is as two convolutions with
kernel size 1.  The dimensionality of input and output is
$d_{\text{model}}=512$, and the inner-layer has dimensionality
$d_{ff}=2048$.

In [21]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        try:
            return self.w_2(self.dropout(self.w_1(x).relu()))
        except RuntimeError:
            return (self.w_2.cuda('cuda:1'))(self.dropout(self.w_1.cuda('cuda:1')(x).relu()))

## Embeddings and Softmax

Similarly to other sequence transduction models, we use learned
embeddings to convert the input tokens and output tokens to vectors
of dimension $d_{\text{model}}$.  We also use the usual learned
linear transformation and softmax function to convert the decoder
output to predicted next-token probabilities.  In our model, we
share the same weight matrix between the two embedding layers and
the pre-softmax linear transformation, similar to
[(cite)](https://arxiv.org/abs/1608.05859). In the embedding layers,
we multiply those weights by $\sqrt{d_{\text{model}}}$.

In [22]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

## Positional Encoding

Since our model contains no recurrence and no convolution, in order
for the model to make use of the order of the sequence, we must
inject some information about the relative or absolute position of
the tokens in the sequence.  To this end, we add "positional
encodings" to the input embeddings at the bottoms of the encoder and
decoder stacks.  The positional encodings have the same dimension
$d_{\text{model}}$ as the embeddings, so that the two can be summed.
There are many choices of positional encodings, learned and fixed
[(cite)](https://arxiv.org/pdf/1705.03122.pdf).

In this work, we use sine and cosine functions of different frequencies:

$$PE_{(pos,2i)} = \sin(pos / 10000^{2i/d_{\text{model}}})$$

$$PE_{(pos,2i+1)} = \cos(pos / 10000^{2i/d_{\text{model}}})$$

where $pos$ is the position and $i$ is the dimension.  That is, each
dimension of the positional encoding corresponds to a sinusoid.  The
wavelengths form a geometric progression from $2\pi$ to $10000 \cdot
2\pi$.  We chose this function because we hypothesized it would
allow the model to easily learn to attend by relative positions,
since for any fixed offset $k$, $PE_{pos+k}$ can be represented as a
linear function of $PE_{pos}$.

In addition, we apply dropout to the sums of the embeddings and the
positional encodings in both the encoder and decoder stacks.  For
the base model, we use a rate of $P_{drop}=0.1$.



In [23]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)


We also experimented with using learned positional embeddings
[(cite)](https://arxiv.org/pdf/1705.03122.pdf) instead, and found
that the two versions produced nearly identical results.  We chose
the sinusoidal version because it may allow the model to extrapolate
to sequence lengths longer than the ones encountered during
training.

## Full Model

> Here we define a function from hyperparameters to a full model.

In [24]:
def make_model_As_Is(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1, use_layers_M_inference = True, skipping_layer_idx = [3,4,5]
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder_MNet_As_Is(EncoderLayer(d_model, c(attn), c(ff), dropout), N, skipping_layer_idx, use_layers_M_inference=use_layers_M_inference),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

def make_model_Compute_M(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1, use_layers_M_inference = True, skipping_layer_idx = [3,4,5]
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder_MNet_Compute_M(EncoderLayer(d_model, c(attn), c(ff), dropout), N, skipping_layer_idx, use_layers_M_inference=use_layers_M_inference),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

def make_model_Base(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1, use_layers_M_inference = True,
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder_Base(EncoderLayer(d_model, c(attn), c(ff), dropout), N, use_layers_M_inference=use_layers_M_inference),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

# Part 2: Model Training

# Training

This section describes the training regime for our models.

## Batches and Masking

In [25]:
class Batch:
    """Object for holding a batch of data with mask during training."""

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask

## Training Loop

In [26]:
class TrainState:
    """Track number of steps, examples, and tokens processed"""

    step: int = 0  # Steps in the current epoch
    accum_step: int = 0  # Number of gradient accumulation steps
    samples: int = 0  # total # of examples used
    tokens: int = 0  # total # of tokens processed

In [27]:
def run_epoch(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()
            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()
            training_loss.append(loss)
        else:
            val_loss.append(loss)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
        
    return total_loss / total_tokens, train_state

In [28]:
def run_epoch_M(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
    use_M=True
):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        # print('batch.src.shape ', batch.src.shape)
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        training_loss.append(loss)
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()


            # update M's ----------------------------------------------
            if use_M:
                model.encoder.mnet1.backwardHidden()


            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1

            scheduler.step()

        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            # print(
            #     (
            #         "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
            #         + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
            #     )
            #     % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            # )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
        
    return total_loss / total_tokens, train_state

## Optimizer

We used the Adam optimizer [(cite)](https://arxiv.org/abs/1412.6980)
with $\beta_1=0.9$, $\beta_2=0.98$ and $\epsilon=10^{-9}$.  We
varied the learning rate over the course of training, according to
the formula:

$$
lrate = d_{\text{model}}^{-0.5} \cdot
  \min({step\_num}^{-0.5},
    {step\_num} \cdot {warmup\_steps}^{-1.5})
$$

This corresponds to increasing the learning rate linearly for the
first $warmup\_steps$ training steps, and decreasing it thereafter
proportionally to the inverse square root of the step number.  We
used $warmup\_steps=4000$.


> Note: This part is very important. Need to train with this setup
> of the model.


> Example of the curves of this model for different model sizes and
> for optimization hyperparameters.

In [29]:
def rate(step, model_size, factor, warmup):
    """
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    """
    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

## Regularization

### Label Smoothing

During training, we employed label smoothing of value
$\epsilon_{ls}=0.1$ [(cite)](https://arxiv.org/abs/1512.00567).
This hurts perplexity, as the model learns to be more unsure, but
improves accuracy and BLEU score.

In [30]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

In [31]:


# def loss(x, crit):
#     d = x + 3 * 1
#     predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d]])
#     return crit(predict.log(), torch.LongTensor([1])).data


# A First  Example

> We can begin by trying out a simple copy-task. Given a random set
> of input symbols from a small vocabulary, the goal is to generate
> back those same symbols.

## Loss Computation

In [32]:
class SimpleLossCompute:
    "A simple loss compute and train function."

    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss

## Greedy Decoding

> This code predicts a translation using greedy decoding for simplicity.

In [33]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

# Part 3: A Real World Example

> Now we consider a real-world example using the Multi30k
> German-English Translation task. This task is much smaller than
> the WMT task considered in the paper, but it illustrates the whole
> system. We also show how to use multi-gpu processing to make it
> really fast.

## Data Loading

> We will load the dataset using torchtext and spacy for
> tokenization.

In [34]:
# Load spacy tokenizer models, download them if they haven't been
# downloaded already


def load_tokenizers():

    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en

In [35]:
def tokenize(text, tokenizer):
    return [tok.text for tok in tokenizer.tokenizer(text)]


def yield_tokens(data_iter, tokenizer, index):
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

In [36]:
def build_vocabulary(spacy_de, spacy_en):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building German Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    print("Building English Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt


def load_vocab(spacy_de, spacy_en):
    if not exists("vocab.pt"):
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Finished.\nVocabulary sizes:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt


if is_interactive_notebook():
    # global variables used later in the script
    spacy_de, spacy_en = show_example(load_tokenizers)
    vocab_src, vocab_tgt = show_example(load_vocab, args=[spacy_de, spacy_en])

Finished.
Vocabulary sizes:
8315
6384


## Iterators

In [37]:
def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
):
    bs_id = torch.tensor([0], device=device)  # <s> token id
    eos_id = torch.tensor([1], device=device)  # </s> token id
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        processed_src = torch.cat(
            [
                bs_id,
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        src_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    max_padding - len(processed_src),
                ),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id,
            )
        )

    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)
    return (src, tgt)

In [38]:
def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=False,
    language_pair = ('de', 'en'),
    dataset_func = datasets.Multi30k
):
    # def create_dataloaders(batch_size=12000):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    train_iter, valid_iter, test_iter = dataset_func(
        language_pair=language_pair
    )

    train_iter_map = to_map_style_dataset(
        train_iter
    )  # DistributedSampler needs a dataset len()
    train_sampler = (
        DistributedSampler(train_iter_map) if is_distributed else None
    )
    valid_iter_map = to_map_style_dataset(valid_iter)
    valid_sampler = (
        DistributedSampler(valid_iter_map) if is_distributed else None
    )

    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=(valid_sampler is None),
        sampler=valid_sampler,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader


def create_dataloaders_test(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=False,
    language_pair = ('de', 'en'),
    dataset_func = datasets.Multi30k
):
    # def create_dataloaders(batch_size=12000):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    train_iter, valid_iter, test_iter = dataset_func(
        language_pair=language_pair
    )

    test_iter_map = to_map_style_dataset(
        test_iter
    )  # DistributedSampler needs a dataset len()
    test_sampler = (
        DistributedSampler(test_iter_map) if is_distributed else None
    )

    test_dataloader = DataLoader(
        test_iter_map,
        batch_size=batch_size,
        shuffle=False,
        sampler=test_sampler,
        collate_fn=collate_fn,
    )

    return test_dataloader

## Training the System

In [39]:
training_loss = []
val_loss = []

def train_worker(
    gpu,
    ngpus_per_node,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    config,
    is_distributed=False,
    model_name = 'base', # base, as_is, compute_m
    dataset_func = datasets.Multi30k,
    language_pair=('de','en'),
    iter_num=-1,
    timestamp=time.time(),
    N = 6,
    skipping_layer_idx = [3,4,5]

):
    time_start = time.time()
    print(f"Train worker process using GPU: {gpu} for training", flush=True)
    torch.cuda.set_device(gpu)

    pad_idx = vocab_tgt["<blank>"]
    d_model = 512

    if model_name == 'base':
        model = make_model_Base(len(vocab_src), len(vocab_tgt), N=N)
    elif model_name == 'as_is':
        model = make_model_As_Is(len(vocab_src), len(vocab_tgt), N=N, skipping_layer_idx = skipping_layer_idx)
    elif model_name == 'compute_m':
        model = make_model_Compute_M(len(vocab_src), len(vocab_tgt), N=N, skipping_layer_idx = skipping_layer_idx)
    else:
        raise NotImplementedError('Model name not implemented: please use one of the following: [base, as_is, compute_m]')

    
    
    model.cuda(gpu)
    module = model
    is_main_process = True
    if is_distributed:
        dist.init_process_group(
            "nccl", init_method="env://", rank=gpu, world_size=ngpus_per_node
        )
        model = DDP(model, device_ids=[gpu])
        module = model.module
        is_main_process = gpu == 0

    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu)

    train_dataloader, valid_dataloader = create_dataloaders(
        gpu,
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=config["batch_size"] // ngpus_per_node,
        max_padding=config["max_padding"],
        is_distributed=is_distributed,
        language_pair=language_pair,
        dataset_func=dataset_func
    )

    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )
    train_state = TrainState()
    best_valid_loss = -999999
    for epoch in range(config["num_epochs"]):
        if is_distributed:
            train_dataloader.sampler.set_epoch(epoch)
            valid_dataloader.sampler.set_epoch(epoch)

        model.train()
        try:
            model.encoder.mnet1.use_M = True 
        except:
            pass
            
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        s = time.time()
        if model_name == 'as_is' or model_name == 'compute_m':
            _, train_state = run_epoch_M(
                (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
                model,
                SimpleLossCompute(module.generator, criterion),
                optimizer,
                lr_scheduler,
                mode="train+log",
                accum_iter=config["accum_iter"],
                train_state=train_state,
                use_M = True,
            )
        else:
            _, train_state = run_epoch( # for base model
                (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
                model,
                SimpleLossCompute(module.generator, criterion),
                optimizer,
                lr_scheduler,
                mode="train+log",
                accum_iter=config["accum_iter"],
                train_state=train_state,
            )
        print(f"Epoch training time: {time.time() - s}")

        GPUtil.showUtilization()
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        if best_valid_loss < 0:
            best_valid_loss = sloss[0]
        elif best_valid_loss > sloss[0]:
            best_valid_loss = sloss[0]
            print('best performance saved')
            file_path = f"%sfinal_best_val_loss_{model_name}_{N}_{timestamp}_trial_{iter_num}.pt" % config["file_prefix"]
            torch.save(module.state_dict(), file_path)
        print(sloss)
        torch.cuda.empty_cache()


    if is_main_process:
        otherinfo = {
            'experiment name' : 'E6D6 - last 3 layers of E are abstracted by M (if not base)',
            'total wallclock training time' : time.time()-time_start,
            'config_info' : config
        }
        file_path = f"%sfinal_{model_name}_{N}_{timestamp}_trial_{iter_num}.pt" % config["file_prefix"]
        torch.save(training_loss, f'saved_{model_name}_{N}_{timestamp}_trial_{iter_num}_train_losses.pt')
        torch.save(val_loss, f'saved_{model_name}_{N}_{timestamp}_trial_{iter_num}_val_losses.pt')
        torch.save(otherinfo, f'saved_{model_name}_{N}_{timestamp}_trial_{iter_num}_otherinfo.pt')
        torch.save(module.state_dict(), file_path)
        file_path1 = f"%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path1)

        del model
        torch.cuda.empty_cache()
        gc.collect()

        return file_path

In [40]:
IS_CUDA = True
H = 8
BATCH_SIZE = 32
MODEL_DIM = 512
def load_trained_model(model_name, iter_num, timestamp, N = 6, skipping_layer_idx = [3,4,5]):
    config = {
        "batch_size": 32,
        "distributed": False,
        "num_epochs": 14,
        "accum_iter": 10,
        "base_lr": 1.0,
        "max_padding": 72,
        "warmup": 3000,
        "file_prefix": "multi30k_model_",
    }
    model_path = "multi30k_model_final.pt"
    if not exists(model_path) or True: # default to always train
        model_fp = train_worker(
        1, 1, vocab_src, vocab_tgt, spacy_de, spacy_en, config, False, model_name=model_name, iter_num=iter_num, timestamp=timestamp,
        N = N, skipping_layer_idx = skipping_layer_idx 
    )
    print(f"Done training {model_name}, config_info={config}")
    return model_fp

s = time.time()


# Pipeline

In [41]:
# run 5 training runs on 1 dataset (for now) and one language pairing (for now de->en), be sure to reference config in load_trained_model to adjust epochs
torch.cuda.empty_cache()
gc.collect()
GPUtil.showUtilization()

num_runs = 5
datasets_list = [datasets.Multi30k]
language_pairings = [('de', 'en')]
model_names = ['base', 'compute_m', 'as_is']

for j in range(len(datasets_list)):
    for lp in language_pairings:
        for mn in model_names:
            print("Model Name", mn, 'trails starts ============================')
            timestamp = 5052023
            for i in range(num_runs):
                print('running', mn, 'N', 6)
                training_loss = []
                val_loss = []
                model_fp = load_trained_model(model_name=mn, iter_num=i, timestamp=timestamp, N = 6, skipping_layer_idx = [3,4,5]) # execute training and save model
                
                print('running', mn, 'N', 3)
                training_loss = []
                val_loss = []
                model_fp = load_trained_model(model_name=mn, iter_num=i, timestamp=timestamp, N = 3, skipping_layer_idx = [2]) # execute training and save model
                
                if mn == 'base':
                    print('running', mn, 'N', 1)
                    training_loss = []
                    val_loss = []
                    model_fp = load_trained_model(model_name=mn, iter_num=i, timestamp=timestamp, N = 1) # execute training and save model
                print(f'Trial: {i}. {mn} model trained successfully see file {model_fp} for model weights')
            print("Model Name", mn, 'trails ends ============================ \n\n')



| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  9% | 30% |
Model Name base trails starts ============================
running base N 6
Train worker process using GPU: 1 for training
[GPU1] Epoch 0 Training ====
Epoch Step:      1 | Accumulation Step:   1 | Loss:   7.63 | Tokens / Sec:  2534.7 | Learning Rate: 5.4e-07
Epoch Step:     41 | Accumulation Step:   5 | Loss:   7.42 | Tokens / Sec:  2549.2 | Learning Rate: 1.1e-05
Epoch Step:     81 | Accumulation Step:   9 | Loss:   7.08 | Tokens / Sec:  2558.8 | Learning Rate: 2.2e-05
Epoch Step:    121 | Accumulation Step:  13 | Loss:   6.71 | Tokens / Sec:  2614.9 | Learning Rate: 3.3e-05
Epoch Step:    161 | Accumulation Step:  17 | Loss:   6.52 | Tokens / Sec:  2607.8 | Learning Rate: 4.4e-05
Epoch Step:    201 | Accumulation Step:  21 | Loss:   6.32 | Tokens / Sec:  2615.8 | Learning Rate: 5.4e-05
Epoch Step:    241 | Accumulation Step:  25 | Loss:   6.20 | Tokens / Sec:  2680.4 | Learning Rate: 6.5e-05
Epoch Step:    2

TypeError: '>' not supported between instances of 'Tensor' and 'TrainState'

# Testing

In [ ]:
stop

NameError: name 'stop' is not defined

In [ ]:
torch.load('./saved_base_3_5032023_trial_1_train_losses.pt')

[tensor(3348.0974, device='cuda:1')]

In [ ]:
# saved_base_6_5032023_train_losses_for.pt
torch.load('./saved_compute_m_6_5032023_train_losses_for.pt')

[tensor(3397.1221, device='cuda:1')]

In [ ]:
def create_dataloaders_test(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=True,
):
    # def create_dataloaders(batch_size=12000):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    _, _, test_iter = datasets.Multi30k(
        language_pair=("de", "en")
    )

    test_iter_map = to_map_style_dataset(
        test_iter
    )  # DistributedSampler needs a dataset len()
    test_sampler = (
        DistributedSampler(test_iter_map) if is_distributed else None
    )


    test_dataloader = DataLoader(
        test_iter_map,
        batch_size=batch_size,
        shuffle=False,
        sampler=test_sampler,
        collate_fn=collate_fn,
    )

    return test_dataloader

In [ ]:
de_refs = []
de_preds = []


def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    # f_preds = open('translation_files/de_preds.txt', 'w+')
    # f_targets = open('translation_files/de_targets.txt', 'w+')
    results = [()] * n_examples
    for idx in range(n_examples):
        print(idx, end=', ')
        # print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        rb.src = rb.src.to('cuda:1')
        rb.src_mask = rb.src_mask.to('cuda:1')

        # print(model.device)
        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        # print(
        #     "Source Text (Input)        : "
        #     + " ".join(src_tokens).replace("\n", "")
        # )
        # print(
        #     "Target Text (Ground Truth) : "
        #     + " ".join(tgt_tokens).replace("\n", "")
        # )
        de_refs.append(" ".join(tgt_tokens).replace("\n", ""))
        # f_targets.write([" ".join(tgt_tokens).replace("\n", "")])

        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        # print("Model Output               : " + model_txt.replace("\n", ""))
        de_preds.append(model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results

In [ ]:
de_refs = []
de_preds = []

model = make_model(len(vocab_src), len(vocab_tgt), N=6, use_layers_M_inference=False)
model.load_state_dict(torch.load("final_saved_data/use_M_as_is_3_layer_M_reran/multi30k_model_final_1683059650.733085.pt"))
model = model.to('cuda:1')

# test_dataloader = create_dataloaders2(
#         torch.device("cpu"),
#         vocab_src,
#         vocab_tgt,
#         spacy_de,
#         spacy_en,
#         batch_size=1,
#         is_distributed=False,
#     )

test_dataloader = create_dataloaders_test(
        torch.device("cuda:1"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )
print(len(test_dataloader))
print("Checking Model Outputs:")
example_data_M = check_outputs(
    test_dataloader, model, vocab_src, vocab_tgt, n_examples=2
)
print(de_refs)
print(de_preds)

compute_bleu(de_refs, de_preds)

Creating MNet_Transformer
1000
Checking Model Outputs:
0, 1, ['<s> Two workers weld the bars of a fence near a busy suburban street . </s>', '<s> A boy with a <unk> nose and writing on his hands is standing in a video rental store . </s>']
['<s> Two workers are <unk> out of a fence on a large , white <unk> street . </s>', '<s> A boy with his nose and <unk> stands in a shop . </s>']


0.5327286

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
# references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
# candidates = [['this', 'is', 'a', 'exam']]
corpus_bleu([de_refs], [de_preds])

0

In [ ]:
PATH = 'final_saved_data/use_M_as_is_3_layer_M_reran/multi30k_model_final_1683059650.733085.pt'

In [ ]:
de_refs = []
de_preds = []

model = make_model(len(vocab_src), len(vocab_tgt), N=6, use_layers_M_inference=False)
model.load_state_dict(torch.load(PATH))
model = model.to('cuda:1')
model.encoder.layers = model.encoder.layers[0:3] 
model.eval()

test_dataloader = create_dataloaders2(
        torch.device("cuda:1"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )
print(len(test_dataloader))

print("Checking Model Outputs:")
example_data_M = check_outputs(
    test_dataloader, model, vocab_src, vocab_tgt, n_examples=len(test_dataloader)
)
print(de_refs)
print(de_preds)

compute_bleu(de_refs, de_preds)

Creating MNet_Transformer
1000
Checking Model Outputs:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 

0.7077451

In [ ]:
fo = open('translation_files/N=6_trained_w_skipping_M_layers_inference_english_preds.txt', 'w')
for sentence in de_preds:
    sentence = (sentence.replace('<s>', '').replace('</s>', '')).strip()
    fo.write("".join(sentence) + '\n')
    print("".join(sentence))
fo.close()

fo = open('translation_files/N=6_trained_w_skipping_M_layers_inference_english_refs.txt', 'w')
for sentence in de_refs:
    sentence = (sentence.replace('<s>', '').replace('</s>', '')).strip()
    fo.write("".join(sentence) + '\n')
    print("".join(sentence))
fo.close()

A man in a gray shirt is jumping over the top of a sand dune .
A child with the number of <unk> of building .
A young lady in white holding a tennis racket .
A man in a pink shirt is sitting on the grass and a ball .
Two people are sitting against a wall at an outdoor table , making faces , and making faces .
A tattooed person looks at a photo at a digital camera or digital camera .
A child looks at <unk> in a store .
Two children cross a small stone bridge over a stone bridge .
An older gentleman and a young girl are having a picnic together together to each other .
A small child in an orange life jacket is holding an oar and paddles on a body of water .
An elderly lady in a green sweatshirt picking out vegetables .
People are admiring art art .
Two girls in shorts holding hands at a pool .
Two men are setting up tools in a workshop .
A guy kissing another guy .
A man in a black leather jacket and cowboy hat strolls on a <unk> street .
A woman is singing in a club with a guitar behind

In [ ]:
de_refs = []
de_preds = []

model = make_model(len(vocab_src), len(vocab_tgt), N=6, use_layers_M_inference=True)
model.load_state_dict(torch.load(PATH))
model = model.to('cuda:1')
model.eval()

test_dataloader = create_dataloaders2(
        torch.device("cuda:1"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

print("Checking Model Outputs:")
example_data_M = check_outputs(
    test_dataloader, model, vocab_src, vocab_tgt, n_examples=len(test_dataloader)
)
print(de_refs)
print(de_preds)

compute_bleu(de_refs, de_preds)

Creating MNet_Transformer
Checking Model Outputs:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 

0.71131897

In [ ]:
fo = open('translation_files/N=6_trained_w_NO_skipping_M_layers_inference_english_preds.txt', 'w')
for sentence in de_preds:
    sentence = (sentence.replace('<s>', '').replace('</s>', '')).strip()
    fo.write("".join(sentence) + '\n')
    print("".join(sentence))
fo.close()

fo = open('translation_files/N=6_trained_w__NO_skipping_M_layers_inference_english_refs.txt', 'w')
for sentence in de_refs:
    sentence = (sentence.replace('<s>', '').replace('</s>', '')).strip()
    fo.write("".join(sentence) + '\n')
    print("".join(sentence))
fo.close()

A cook is posing for the camera .
A man with red - hair is sitting and playing an acoustic guitar .
A girl in white and green walk past a blue <unk> shop .
A beautiful woman is playing on a <unk> .
A man and a woman are sitting by a fence talking .
A man walking in a <unk> - boat in the water .
A child in a red coat and hat is holding a large pile of snow .
Two men play guitar in front of a large audience .
Children are fighting for the ball .
A young woman with a purple cloth over her face is relaxing on a wooden deck .
A man selling goods on the edge of the street in a large mountain .
Two boys play on a bike .
Two construction workers are putting hay over some metal .
<unk> and <unk> are in the city of <unk> in the city .
A lady in a red sweater and jeans sits with her hands on her left left left hand .
The yellow dog carries a stick across the water .
A man stands on a rocky cliff overlooking a body of water .
A group of men standing in a field of black balls .
An oriental worker i

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
candidates = [['this', 'is', 'a', 'exam']]
score = corpus_bleu(references, candidates)
print(score)

8.636168555094496e-78


/home/kyle/pythonenvs/venv38/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
print(de_refs)
print(de_preds)

[['<s> An indian man walks past a temple on his way to pray . </s>'], ['<s> A man in a blue shirt is holding a sign that says " <unk> on now <unk> what \'s <unk> than tea . " </s>']]
['<s> An Indian man walks past the end of a temple . </s>', '<s> A man in a blue shirt holds a sign that says " <unk> <unk> <unk> " . </s>']


In [ ]:
stop

In [ ]:
stop

NameError: name 'stop' is not defined

# Additional Components: BPE, Search, Averaging


> So this mostly covers the transformer model itself. There are four
> aspects that we didn't cover explicitly. We also have all these
> additional features implemented in
> [OpenNMT-py](https://github.com/opennmt/opennmt-py).




> 1) BPE/ Word-piece: We can use a library to first preprocess the
> data into subword units. See Rico Sennrich's
> [subword-nmt](https://github.com/rsennrich/subword-nmt)
> implementation. These models will transform the training data to
> look like this:

▁Die ▁Protokoll datei ▁kann ▁ heimlich ▁per ▁E - Mail ▁oder ▁FTP
▁an ▁einen ▁bestimmte n ▁Empfänger ▁gesendet ▁werden .


> 2) Shared Embeddings: When using BPE with shared vocabulary we can
> share the same weight vectors between the source / target /
> generator. See the [(cite)](https://arxiv.org/abs/1608.05859) for
> details. To add this to the model simply do this:

In [ ]:
if False:
    model.src_embed[0].lut.weight = model.tgt_embeddings[0].lut.weight
    model.generator.lut.weight = model.tgt_embed[0].lut.weight


> 3) Beam Search: This is a bit too complicated to cover here. See the
> [OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py/)
> for a pytorch implementation.
>



> 4) Model Averaging: The paper averages the last k checkpoints to
> create an ensembling effect. We can do this after the fact if we
> have a bunch of models:

In [ ]:
def average(model, models):
    "Average models into model"
    for ps in zip(*[m.params() for m in [model] + models]):
        ps[0].copy_(torch.sum(*ps[1:]) / len(ps[1:]))

# Results

On the WMT 2014 English-to-German translation task, the big
transformer model (Transformer (big) in Table 2) outperforms the
best previously reported models (including ensembles) by more than
2.0 BLEU, establishing a new state-of-the-art BLEU score of
28.4. The configuration of this model is listed in the bottom line
of Table 3. Training took 3.5 days on 8 P100 GPUs. Even our base
model surpasses all previously published models and ensembles, at a
fraction of the training cost of any of the competitive models.

On the WMT 2014 English-to-French translation task, our big model
achieves a BLEU score of 41.0, outperforming all of the previously
published single models, at less than 1/4 the training cost of the
previous state-of-the-art model. The Transformer (big) model trained
for English-to-French used dropout rate Pdrop = 0.1, instead of 0.3.


![](images/results.png)



> With the addtional extensions in the last section, the OpenNMT-py
> replication gets to 26.9 on EN-DE WMT. Here I have loaded in those
> parameters to our reimplemenation.

In [ ]:
# Load data and model for output checks

In [ ]:
de_refs = []
de_preds = []


def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    results = [()] * n_examples
    for idx in range(n_examples):
        print(idx, end=', ')
        # print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        rb.src = rb.src.to('cuda:1')
        rb.src_mask = rb.src_mask.to('cuda:1')

        # print(model.device)
        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        # print(
        #     "Source Text (Input)        : "
        #     + " ".join(src_tokens).replace("\n", "")
        # )
        # print(
        #     "Target Text (Ground Truth) : "
        #     + " ".join(tgt_tokens).replace("\n", "")
        # )
        de_refs.append([" ".join(tgt_tokens).replace("\n", "")])
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        # print("Model Output               : " + model_txt.replace("\n", ""))
        de_preds.append(model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results


def run_model_example(n_examples=100):
    global vocab_src, vocab_tgt, spacy_de, spacy_en

    print("Preparing Data ...")
    _, valid_dataloader = create_dataloaders(
        torch.device("cuda:1"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

    # n_examples = len(iter(valid_dataloader))
    n_examples = 10

    print("Loading Trained Model ...")

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(
        torch.load("multi30k_model_final.pt", map_location=torch.device("cuda:1"))
    )
    model = model.to('cuda:1')

    print("Checking Model Outputs:")
    example_data = check_outputs(
        valid_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples
    )
    return model, example_data


execute_example(run_model_example)

Preparing Data ...
Loading Trained Model ...
Creating MNet_Transformer
Checking Model Outputs:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

In [ ]:
_, valid_dataloader = create_dataloaders(
        torch.device("cpu"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

len(iter(valid_dataloader))

1015

In [ ]:
de_preds[:10], de_refs[:10]

(['<s> A group of <unk> holding the <unk> of <unk> and standing on the beach overlooking water . </s>',
  '<s> A man is walking down the sidewalk next to a city street . </s>',
  '<s> A man is running through the snow with his mountains . </s>',
  '<s> People are walking on a sidewalk in front of a <unk> city street . </s>',
  '<s> Two women , one in a <unk> , are performing a move , on a mat . </s>',
  '<s> There is a man , a man in a blanket , wearing a bright colored jacket , and is riding a snowy hill . </s>',
  '<s> A little baby wearing pink , in her dark clothing , is laying down . </s>',
  '<s> A group of Asian men are running in a marathon . </s>',
  '<s> Two <unk> , one in red and the other wearing red , are being <unk> by a crowd . </s>',
  '<s> Two girls are jumping and one woman is holding a sign behind them . </s>'],
 [['<s> A group of beach - goers are holding hands forming a line facing the water , down the beach . </s>'],
  ['<s> A man is walking down the sidewalk next

## BLEU score (corpus level)

In [ ]:
# !pip install nltk

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
corpus_bleu(de_refs, de_preds)

0.35154968464064296

## Attention Visualization

> Even with a greedy decoder the translation looks pretty good. We
> can further visualize it to see what is happening at each layer of
> the attention

In [ ]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    "convert a dense matrix to a data frame with row and column indices"
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s"
                % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s"
                % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        # if float(m[r,c]) != 0 and r < max_row and c < max_col],
        columns=["row", "column", "value", "row_token", "col_token"],
    )


def attn_map(attn, layer, head, row_tokens, col_tokens, max_dim=30):
    df = mtx2df(
        attn[0, head].data,
        max_dim,
        max_dim,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        .properties(height=400, width=400)
        .interactive()
    )

In [ ]:
def get_encoder(model, layer):
    return model.encoder.layers[layer].self_attn.attn


def get_decoder_self(model, layer):
    return model.decoder.layers[layer].self_attn.attn


def get_decoder_src(model, layer):
    return model.decoder.layers[layer].src_attn.attn


def visualize_layer(model, layer, getter_fn, ntokens, row_tokens, col_tokens):
    # ntokens = last_example[0].ntokens
    attn = getter_fn(model, layer)
    n_heads = attn.shape[1]
    charts = [
        attn_map(
            attn,
            0,
            h,
            row_tokens=row_tokens,
            col_tokens=col_tokens,
            max_dim=ntokens,
        )
        for h in range(n_heads)
    ]
    assert n_heads == 8
    return alt.vconcat(
        charts[0]
        # | charts[1]
        | charts[2]
        # | charts[3]
        | charts[4]
        # | charts[5]
        | charts[6]
        # | charts[7]
        # layer + 1 due to 0-indexing
    ).properties(title="Layer %d" % (layer + 1))

## Encoder Self Attention

In [ ]:
def viz_encoder_self():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[
        len(example_data) - 1
    ]  # batch object for the final example

    layer_viz = [
        visualize_layer(
            model, layer, get_encoder, len(example[1]), example[1], example[1]
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        # & layer_viz[1]
        & layer_viz[2]
        # & layer_viz[3]
        & layer_viz[4]
        # & layer_viz[5]
    )


show_example(viz_encoder_self)

Preparing Data ...
Loading Trained Model ...
Creating MNet_Transformer
Checking Model Outputs:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 

## Decoder Self Attention

In [ ]:
def viz_decoder_self():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[len(example_data) - 1]

    layer_viz = [
        visualize_layer(
            model,
            layer,
            get_decoder_self,
            len(example[1]),
            example[1],
            example[1],
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        & layer_viz[1]
        & layer_viz[2]
        & layer_viz[3]
        & layer_viz[4]
        & layer_viz[5]
    )


show_example(viz_decoder_self)

Preparing Data ...


Loading Trained Model ...


Checking Model Outputs:

Example 0 ========



Source Text (Input)        : <s> Drei Menschen wandern auf einem stark verschneiten Weg . </s>
Target Text (Ground Truth) : <s> A <unk> of people are hiking throughout a heavily snowed path . </s>


Model Output               : <s> Three people hiking on a busy <unk> . </s>


alt.HConcatChart(...)

## Decoder Src Attention

In [ ]:
def viz_decoder_src():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[len(example_data) - 1]

    layer_viz = [
        visualize_layer(
            model,
            layer,
            get_decoder_src,
            max(len(example[1]), len(example[2])),
            example[1],
            example[2],
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        & layer_viz[1]
        & layer_viz[2]
        & layer_viz[3]
        & layer_viz[4]
        & layer_viz[5]
    )


show_example(viz_decoder_src)

Preparing Data ...


Loading Trained Model ...


Checking Model Outputs:

Example 0 ========



Source Text (Input)        : <s> Baby sieht sich die Blätter am Zweig eines Baumes an . </s>
Target Text (Ground Truth) : <s> Baby looking at the leaves on a branch of a tree . </s>


Model Output               : <s> A baby is looking at the leaves at a tree . </s>


alt.HConcatChart(...)

# Conclusion

 Hopefully this code is useful for future research. Please reach
 out if you have any issues.


 Cheers,
 Sasha Rush, Austin Huang, Suraj Subramanian, Jonathan Sum, Khalid Almubarak,
 Stella Biderman